In [1]:
import torch
import torch.nn as nn
from torch import optim

from matplotlib import pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import os
from sys import platform

from torch.utils.data import TensorDataset, DataLoader

In [2]:
device = "cpu"
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.backends.mps.is_available():
    device = torch.device("mps")
torch.set_default_device(device)
print("Device in use :", device)

Device in use : mps


In [3]:
BATCH_SIZE = 24
EPOCHS = 10

In [4]:
train_pcs = np.load("../data/ModelNet10/train_pc.npz")
test_pcs = np.load("../data/ModelNet10/test_pc.npz")

train_data, train_labels = train_pcs['train_pc'], train_pcs["train_labels"]
test_data, test_labels = test_pcs['test_pc'], test_pcs["test_labels"]

# Lookup table to encode Labels to be consumed by the model
lookup = {
    "bed": [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
    "bathtub": [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    "chair": [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
    "desk": [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
    "dresser": [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
    "monitor": [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
    "night_stand": [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
    "sofa": [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
    "table": [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
    "toilet": [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
}

train_labels = np.array([lookup[i] for i in train_labels]).astype(np.float32)
test_labels = np.array([lookup[i] for i in test_labels]).astype(np.float32)

In [5]:
class CustomMax(nn.Module):
    def forward(self, inp):
        return torch.max(inp, dim = 1).values

class PointNet(nn.Module):
    def __init__(self, num_points, batch_size, channels = 3, n_classes = 10):
        super().__init__()
        self.channels = channels
        self.n_classes = n_classes
        self.num_points = num_points
        self.batch_size = batch_size
        self.model = nn.Sequential(
            self.__dense_layer_op(),
            self.__dense_layer_op(32, 32),
            self.__dense_layer_op(32, 32),
            self.__dense_layer_op(32, 64),
            self.__dense_layer_op(64, 512),
            # At this point we have batch_size x 2048 x 512 features
            # We want batch_size x 512 feature ahead of this point
            # So we want to perform global max pooling over the 2048 x 512 set for each example.
            CustomMax(),
            self.__dense_layer_op(512, 256, over_out_channel=True),
            nn.Dropout(0.3),
            self.__dense_layer_op(256, 128, over_out_channel=True),
            nn.Dropout(0.3),
            nn.Linear(128, self.n_classes),
            nn.Softmax()
        )
    
    def __dense_layer_op(self, units_in = 3, units_out = 32, over_out_channel = False):
        if over_out_channel:
            return nn.Sequential(
            nn.Linear(units_in, units_out),
            nn.BatchNorm1d(units_out, momentum=0.7),
            nn.LeakyReLU(0.2)
        )
        
        return nn.Sequential(
            nn.Linear(units_in, units_out),
            nn.BatchNorm1d(self.num_points, momentum=0.7),
            nn.LeakyReLU(0.2)
        )
    
    def forward(self, inp):
        return self.model(inp)

In [6]:
class PtNet_Model:
    def __init__(self, num_points, n_classes, batch_size, epochs, device):
        self.num_points = num_points
        self.batch_size = batch_size
        self.epochs = epochs
        self.n_classes = n_classes
        self.learning_rate=0.002
        self.device = device
        self.channels = 4

        self.PtNet = PointNet(self.num_points, self.channels, self.n_classes).to(self.device)

        # Setup optimizers
        self.optimizer = optim.Adam(self.PtNet.parameters(), lr=self.learning_rate)

        # Setup loss function
        self.loss_fn = torch.nn.CrossEntropyLoss()
    
    def train(self,):
        global train_data
        global train_labels

        train_dataset = TensorDataset(torch.from_numpy(train_data), torch.from_numpy(train_labels))
        idcs = np.arange(len(train_data))
        np.random.shuffle(idcs)
        validation_dataset = torch.utils.data.Subset(train_dataset, idcs[:350])

        train_loader = DataLoader(dataset = train_dataset, batch_size = self.batch_size, shuffle=True, generator=torch.Generator(device=self.device), pin_memory=True)
        # train_loader = DataLoader(dataset = train_dataset, batch_size = self.batch_size, shuffle=True)
        validation_loader = DataLoader(dataset = validation_dataset, shuffle=True, batch_size = self.batch_size, generator=torch.Generator(device=self.device), pin_memory=True)

        for epoch in tqdm(range(1, self.epochs + 1)):
            for batch_idx, (data, labels) in enumerate(train_loader):
                data, labels = data.to(self.device), labels.to(self.device)
                model_labels = self.PtNet(data)
                loss = self.loss_fn(model_labels, labels)
                
                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()
                # if batch_idx % 100==0:
                #     print("Loss :", loss.item())

            # Perform validation operation
            with torch.no_grad():
                correct, total = 0, 0
                for batch_idx, (data, labels) in enumerate(validation_loader):
                    data, labels = data.to(self.device), labels.to(self.device)
                    outputs = self.PtNet(data)
                    loss = self.loss_fn(outputs, labels)
                    predicted = torch.argmax(outputs.data, dim=1)
                    labels = torch.argmax(labels, dim=1)
                    
                    total += labels.size(0)
                    correct += (predicted == labels).sum().item()
                print(f'Validation Accuracy of the network : {100 * correct // total}% and validation loss : {np.round(loss.item(), 3)}')
    def test(self,):
        global test_data
        global test_labels

        test_dataset = TensorDataset(torch.from_numpy(test_data), torch.from_numpy(test_labels))
        test_loader = DataLoader(dataset=test_dataset, shuffle=True, batch_size=100)

        with torch.no_grad():
            correct, total = 0, 0
            loss = 0.0
            for batch_idx, (data, labels) in enumerate(test_loader):
                data, labels = data.to(self.device), labels.to(self.device)
                outputs = self.PtNet(data)
                loss = self.loss_fn(outputs, labels)
                _, predicted = torch.max(outputs.data, 1)
                _, labels = torch.max(labels, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
            print(f'Test Accuracy of the network : {100 * correct // total}% and test loss : {np.round(loss.item(), 3)}')


In [7]:
pt_net = PtNet_Model(2048, 10, BATCH_SIZE, EPOCHS, device = device)

In [8]:
pt_net.train()

  0%|          | 0/10 [00:00<?, ?it/s]

/Users/anmol/Codebase/Environments/AI/lib/python3.11/site-packages/torch/nn/functional.py:1881: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return handle_torch_function(softmax, (input,), input, dim=dim, _stacklevel=_stacklevel, dtype=dtype)


Validation Accuracy of the network : 71% and validation loss : 1.81
Validation Accuracy of the network : 73% and validation loss : 1.756
Validation Accuracy of the network : 75% and validation loss : 1.627
Validation Accuracy of the network : 77% and validation loss : 1.889
Validation Accuracy of the network : 78% and validation loss : 1.585
Validation Accuracy of the network : 78% and validation loss : 1.728
Validation Accuracy of the network : 78% and validation loss : 1.583
Validation Accuracy of the network : 77% and validation loss : 1.726
Validation Accuracy of the network : 81% and validation loss : 1.799
Validation Accuracy of the network : 80% and validation loss : 1.659


In [9]:
if platform == "darwin":
    os.system("say 'Model Training DONE!'")